# Notebook for simulating, saving and analysing MOMA-like data for a parameter range

In [1]:
from cells_simulation import *

## Simulation with parameter range

In [3]:
# ========== Simulation parameters ========== #
out_dir = mk_mising_dir('../data/simulation_set')
save_dataset = True    # saves data set as csv
run_ggp = True         # runs the ggp code (located in "../bin/" relative to this nb)

dt = 1e-3
dt_measument = 3 # in minutes
n_cells = 250 # number of cells that will be simulated

div_mode = "sizer"
division_log_length = 1+np.log(2)   # for sizer: division, when log_length hits division_log_length
division_time = 60 - 1e-10          # for timer: division, when cell cycle time hits division_time
division_addition = np.log(2)       # for adder: divsion, when division_addition in log_length was added in cell cycle

for i in range(10):
    # ========== Model parameters ========== #
    parameter_set = {   "mean_lambda": 0.01, 
                        "gamma_lambda": 0.01,
                        "var_lambda": 1e-7,
                        "mean_q": 10,
                        "gamma_q": 1e-2,
                        "var_q": 0.1,
                        "beta": 0.001, #1e-3,
                        "var_x": 1e-3,
                        "var_g": 5000,
                        "var_dx": 0.001,
                        "var_dg": 500}

    # --------- SIMULATE --------- #
    cells_simulated = simulate_cells(dt, n_cells, parameter_set, div_mode,
                         division_log_length, 
                         division_time, 
                         division_addition)

    dataset = build_data_set(cells_simulated, parameter_set['var_x'], parameter_set['var_g'], int(dt_measument/dt))

    # ----------- SAVE ----------- #
    if save_dataset:
        directory, filename = get_next_file_name(out_dir)

        fuzzy_params = add_noise2params(parameter_set, 0.2)
        write_param_file(os.path.join(directory, "parameters.txt"), fuzzy_params, {"var_dg" : ["free"]}) 

        write_csv_config(os.path.join(directory, "csv_config.txt"))
        dataset.to_csv(filename)
        print("New simulation saved in", filename)

        # ----------- RUN ----------- #
        if run_ggp:
            run_cmd = suggest_run_command(directory, filename, modes="-m -p")
            print("run: -> \n", run_cmd)
            s = subprocess.run(run_cmd, capture_output=False, shell=True)
            print(s)
        

| ==================== | Progress 100%  No of cells:  250
Every 3000 th datapoint is saved
New simulation saved in ../data/simulation_set/simulation_set_0/simulation_set_0.csv
run: -> 
 ../bin/gfp_gaussian -c ../data/simulation_set/simulation_set_0/csv_config.txt -b ../data/simulation_set/simulation_set_0/parameters.txt -r 1e-3  -i ../data/simulation_set/simulation_set_0/simulation_set_0.csv -l 0 -m -p
CompletedProcess(args='../bin/gfp_gaussian -c ../data/simulation_set/simulation_set_0/csv_config.txt -b ../data/simulation_set/simulation_set_0/parameters.txt -r 1e-3  -i ../data/simulation_set/simulation_set_0/simulation_set_0.csv -l 0 -m -p', returncode=0)
| ==================== | Progress 100%  No of cells:  250
Every 3000 th datapoint is saved
New simulation saved in ../data/simulation_set/simulation_set_1/simulation_set_1.csv
run: -> 
 ../bin/gfp_gaussian -c ../data/simulation_set/simulation_set_1/csv_config.txt -b ../data/simulation_set/simulation_set_1/parameters.txt -r 1e-3  -i .